In [1]:
import sys, os, subprocess
import math, time, threading
from urllib.parse import urlencode

pkgs = ["flask", "pyngrok", "folium"]
missing = []
for p in pkgs:
    try:
        __import__(p)
    except ImportError:
        missing.append(p)
if missing:
    subprocess.run([sys.executable, "-m", "pip", "install", *missing], check=True)

from flask import Flask, redirect, url_for, request, render_template_string
from pyngrok import ngrok, conf
import folium

REFERENCE_POINTS = {
    "sol": (40.41694, -3.70361),
    "gran via": (40.42030, -3.70577),
    "retiro": (40.41526, -3.68442),
    "atocha": (40.40758, -3.69194),
    "ie tower": (40.47778, -3.68889),
}
spots = [
    {"id": 1, "name": "Spot 1", "location": (40.4785, -3.6885), "status": "available", "type": "electric"},
    {"id": 2, "name": "Spot 2", "location": (40.4765, -3.6890), "status": "available", "type": "truck"},
    {"id": 3, "name": "Spot 3", "location": (40.4795, -3.6870), "status": "available", "type": "electric"},
    {"id": 4, "name": "Spot 4", "location": (40.4770, -3.6900), "status": "booked", "type": "truck"},
    {"id": 5, "name": "Spot 5", "location": (40.4790, -3.6900), "status": "available", "type": "electric"},
    {"id": 6, "name": "Spot 6", "location": (40.4790, -3.6875), "status": "available", "type": "truck"},
    {"id": 7, "name": "Spot 7", "location": (40.4765, -3.6875), "status": "available", "type": "electric"},
    {"id": 8, "name": "Spot 8", "location": (40.4765, -3.6905), "status": "available", "type": "truck"},
    {"id": 9, "name": "Spot 9", "location": (40.4803, -3.6889), "status": "available", "type": "regular"},
    {"id": 10, "name": "Spot 10", "location": (40.4753, -3.6889), "status": "available", "type": "regular"},
    {"id": 11, "name": "Sol P1", "location": (40.4175, -3.7045), "status": "available", "type": "regular"},
    {"id": 12, "name": "Sol P2", "location": (40.4164, -3.7022), "status": "available", "type": "electric"},
    {"id": 13, "name": "Gran Via P1", "location": (40.4211, -3.7042), "status": "available", "type": "regular"},
    {"id": 14, "name": "Gran Via P2", "location": (40.4198, -3.7069), "status": "booked", "type": "electric"},
    {"id": 15, "name": "Retiro P1", "location": (40.4159, -3.6830), "status": "available", "type": "truck"},
    {"id": 16, "name": "Retiro P2", "location": (40.4144, -3.6861), "status": "available", "type": "regular"},
    {"id": 17, "name": "Atocha P1", "location": (40.4079, -3.6932), "status": "available", "type": "regular"},
    {"id": 18, "name": "Atocha P2", "location": (40.4067, -3.6905), "status": "available", "type": "electric"},
    {"id": 19, "name": "Salamanca P1","location": (40.4780, -3.6840), "status": "available", "type": "truck"},
    {"id": 20, "name": "Salamanca P2","location": (40.4790, -3.6855), "status": "booked", "type": "regular"},
]
last_booked = None

def haversine_km(a, b):
    R = 6371.0
    lat1, lon1 = a; lat2, lon2 = b
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi, dlmb = math.radians(lat2 - lat1), math.radians(lon2 - lon1)
    x = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2*R*math.atan2(math.sqrt(x), math.sqrt(1-x))

def find_spots(origin, spot_type=None, max_meters=None):
    out = []
    for s in spots:
        if s["status"] != "available": continue
        if spot_type and s["type"].lower() != spot_type.lower(): continue
        d_km = haversine_km(origin, s["location"])
        if max_meters is not None and d_km*1000 > max_meters: continue
        out.append({**s, "distance_km": d_km})
    out.sort(key=lambda x: (x["distance_km"], x["id"]))
    return out

app = Flask(__name__)

@app.after_request
def no_cache(resp):
    resp.headers["Cache-Control"] = "no-store, no-cache, must-revalidate, max-age=0"
    resp.headers["Pragma"] = "no-cache"
    resp.headers["Expires"] = "0"
    return resp

SAFE_PARAMS = {"ref", "lat", "lon", "type", "max_m", "mode", "k", "use_custom"}
def keep_params(args):
    return {k: str(args.get(k)) for k in SAFE_PARAMS if args.get(k) is not None}

def parse_origin(args):
    ref = (args.get("ref") or "ie tower").lower()
    use_custom = args.get("use_custom") == "1"
    if use_custom:
        lat, lon = args.get("lat"), args.get("lon")
        try:
            return (float(lat), float(lon)), f"custom ({lat}, {lon})", ref, lat, lon, use_custom
        except (ValueError, TypeError):
            pass
    if ref in REFERENCE_POINTS:
        return REFERENCE_POINTS[ref], ref, ref, None, None, use_custom
    return REFERENCE_POINTS["ie tower"], "ie tower", "ie tower", None, None, use_custom

def parse_filters(args):
    t = args.get("type"); t = t if t in ("regular","electric","truck") else None
    try: m = float(args.get("max_m","")) if args.get("max_m") else None
    except ValueError: m = None
    try: k = int(args.get("k", 5))
    except ValueError: k = 5
    mode = args.get("mode", "nearest"); mode = mode if mode in ("nearest","list") else "nearest"
    return t, m, k, mode

@app.route("/")
def index():
    origin, origin_label, sel_ref, lat, lon, use_custom = parse_origin(request.args)
    spot_type, max_m, k, mode = parse_filters(request.args)
    rows = find_spots(origin, spot_type, max_m)
    match_ids = {r["id"] for r in rows}
    m = folium.Map(location=origin, zoom_start=15)
    folium.Marker(origin, tooltip=f"You are here ({origin_label})",
                  icon=folium.Icon(color="purple", icon="user", prefix="fa")).add_to(m)
    highlight_ids = []
    if last_booked is not None:
        highlight_ids = [last_booked]
    elif rows:
        highlight_ids = [rows[0]["id"]] if mode == "nearest" else [r["id"] for r in rows[:k]]
    clean_qs = keep_params(request.args)
    for s in spots:
        if s["status"] == "available" and s["id"] not in match_ids and last_booked is None:
            continue
        status = s["status"]
        is_highlight = (s["id"] in highlight_ids) and (status == "available" or s["id"] == last_booked)
        if status == "available":
            icon = folium.Icon(
                color=("white" if is_highlight else {"electric":"green","truck":"blue"}.get(s["type"],"orange")),
                icon=("star" if is_highlight else ("bolt" if s["type"]=="electric" else ("truck" if s["type"]=="truck" else "car"))),
                icon_color=("orange" if is_highlight else None),
                prefix="fa",
            )
            book_url = url_for("book_spot", spot_id=s["id"]) + ("?" + urlencode(clean_qs) if clean_qs else "")
            popup = (f"<b>{s['name']}</b><br>Type: {s['type'].capitalize()}<br>"
                     f"<a class='pf-action' href='{book_url}' target='_self' rel='noopener'>Book this spot</a>")
            tooltip = f"{s['name']} ({s['type'].capitalize()} – Available)"
        else:
            mine = (s["id"] == (last_booked or -1))
            icon = folium.Icon(color=("purple" if mine else "red"),
                               icon=("bolt" if s["type"]=="electric" else ("truck" if s["type"]=="truck" else "car")),
                               prefix="fa")
            if mine:
                cancel_url = url_for("cancel_spot", spot_id=s["id"]) + ("?" + urlencode(clean_qs) if clean_qs else "")
                popup = (f"<b>{s['name']}</b><br>Type: {s['type'].capitalize()}<br>"
                         f"Status: Booked (Your reservation)"
                         f"<br><a class='pf-action' href='{cancel_url}' target='_self' rel='noopener'>Cancel reservation</a>")
                tooltip = f"{s['name']} (Booked – Yours)"
            else:
                popup = f"<b>{s['name']}</b><br>Type: {s['type'].capitalize()}<br>Status: Booked"
                tooltip = f"{s['name']} (Booked)"
        folium.Marker(location=s["location"], tooltip=tooltip, popup=popup, icon=icon).add_to(m)
    for sid in highlight_ids:
        t = next((x for x in spots if x["id"] == sid), None)
        if t:
            folium.PolyLine([origin, t["location"]], color="orange", weight=5, opacity=0.6).add_to(m)
    html_map = m._repr_html_()
    html = """
    <!doctype html>
    <html>
    <head>
      <meta charset="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/>
      <title>ParkFinder</title>
      <style>
        *{box-sizing:border-box} body{font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif;margin:0}
        header{padding:12px 16px;background:#111827;color:#fff;display:flex;align-items:center;justify-content:space-between}
        .wrap{display:grid;grid-template-columns:320px 1fr;min-height:calc(100vh - 48px)}
        form{padding:12px 16px;border-right:1px solid #e5e7eb}
        label{display:block;font-size:12px;color:#374151;margin-top:10px}
        input,select{width:100%;padding:8px;margin-top:4px}
        .row{display:flex;gap:8px}.row>div{flex:1}
        button{margin-top:12px;padding:10px 12px;width:100%;cursor:pointer}
        .hint{font-size:12px;color:#6b7280;margin-top:6px}
        .map{height:calc(100vh - 48px);overflow:hidden}
        fieldset{border:1px solid #e5e7eb;border-radius:8px;padding:8px;margin-top:10px}
        legend{font-size:12px;color:#6b7280;padding:0 6px}
        a.pf-action{color:#2563eb;text-decoration:underline;cursor:pointer}
      </style>

      <script>
        async function pfNavigate(url) {
          try {
            const res = await fetch(url, { headers: { "X-Requested-With": "fetch" } });
            const text = await res.text();
            const doc = new DOMParser().parseFromString(text, "text/html");
            const nextRoot = doc.querySelector("#app-root");
            const curRoot  = document.querySelector("#app-root");
            if (nextRoot && curRoot) {
              curRoot.replaceWith(nextRoot);
            } else {
              window.location.href = url;
            }
          } catch (e) {
            window.location.href = url;
          }
        }

        function installRouter(scope=document) {
          scope.addEventListener("click", (ev) => {
            const a = ev.target.closest("a.pf-action");
            if (!a) return;
            const url = a.getAttribute("href");
            if (!url) return;
            ev.preventDefault();
            pfNavigate(url);
          });
        }

        document.addEventListener("DOMContentLoaded", () => {
          installRouter(document);
        });
      </script>
    </head>
    <body>
      <div id="app-root">
        <header><strong>ParkFinder</strong><span>Filters → Map updates instantly</span></header>
        <div class="wrap">
          <form method="get" action="/">
            <label>Reference point</label>
            <select name="ref">
              {% for key in refs %}
                <option value="{{key}}" {% if key==sel_ref %}selected{% endif %}>{{key}}</option>
              {% endfor %}
            </select>

            <fieldset>
              <legend>Custom coordinates</legend>
              <label style="display:flex;align-items:center;gap:8px;margin-top:0">
                <input id="use_custom" type="checkbox" name="use_custom" value="1" {% if use_custom %}checked{% endif %}/>
                <span>Use custom coordinates</span>
              </label>
              <div class="row">
                <div><label>Latitude</label><input name="lat" value="{{lat or ''}}" placeholder="40.4169"/></div>
                <div><label>Longitude</label><input name="lon" value="{{lon or ''}}" placeholder="-3.7035"/></div>
              </div>
            </fieldset>

            <label>Type</label>
            <select name="type">
              <option value="">Any</option>
              {% for t in ['regular','electric','truck'] %}
                <option value="{{t}}" {% if sel_type==t %}selected{% endif %}>{{t}}</option>
              {% endfor %}
            </select>
            <label>Max distance (meters)</label>
            <input type="number" name="max_m" min="0" step="50" value="{{max_m or ''}}"/>
            <label>Mode</label>
            <select name="mode">
              <option value="nearest" {% if mode=='nearest' %}selected{% endif %}>Nearest only</option>
              <option value="list" {% if mode=='list' %}selected{% endif %}>Top-K list</option>
            </select>
            <label>Top K (when list)</label>
            <input type="number" name="k" min="1" max="20" value="{{k}}"/>
            <button type="submit">Update map</button>
            <div class="hint">Origin: <strong>{{origin_label}}</strong> • Matches: {{rows|length}}</div>
          </form>

          <div class="map" id="map-container">{{ html_map | safe }}</div>
        </div>
      </div>
    </body>
    </html>
    """

    return render_template_string(
        html,
        refs=sorted(REFERENCE_POINTS.keys()),
        sel_ref=sel_ref, lat=lat, lon=lon, use_custom=use_custom,
        sel_type=spot_type, max_m=max_m, mode=mode, k=k,
        origin_label=origin_label, rows=rows, html_map=html_map
    )

@app.route("/book/<int:spot_id>")
def book_spot(spot_id):
    global last_booked
    if last_booked and last_booked != spot_id:
        for s in spots:
            if s["id"] == last_booked:
                s["status"] = "available"; break
        last_booked = None
    for s in spots:
        if s["id"] == spot_id and s["status"] == "available":
            s["status"] = "booked"; last_booked = spot_id; break
    clean_qs = keep_params(request.args)
    return redirect(url_for("index", **clean_qs, _=int(time.time())), code=303)

@app.route("/cancel/<int:spot_id>")
def cancel_spot(spot_id):
    global last_booked
    for s in spots:
        if s["id"] == spot_id and s["status"] == "booked":
            s["status"] = "available"
            if last_booked == spot_id: last_booked = None
            break
    clean_qs = keep_params(request.args)
    return redirect(url_for("index", **clean_qs, _=int(time.time())), code=303)

env = "script"
try:
    from IPython import get_ipython
    if get_ipython(): env = "jupyter"
except ImportError:
    env = "script"
if "google.colab" in sys.modules: env = "colab"
elif "KAGGLE_URL_BASE" in os.environ or os.path.exists("/kaggle"): env = "kaggle"

if env in ("colab","kaggle"):
    from IPython.display import HTML, display, clear_output
    clear_output(wait=True)
    conf.get_default().auth_token = "35JE4f4YzGObTuQkCdcXufzOWSd_8mLqeByHADYjqgTQcYY7"
    try: ngrok.kill()
    except: pass
    try:
        for t in ngrok.get_tunnels():
            try: ngrok.disconnect(t.public_url)
            except: pass
    except: pass
    public_url = ngrok.connect(5000, bind_tls=True).public_url
    print(f"* ngrok tunnel: {public_url}")
    th = threading.Thread(target=app.run, kwargs={"use_reloader": False, "port": 5000, "host": "0.0.0.0"}, daemon=True)
    th.start()
    display(HTML(f"<iframe src='{public_url}' width='100%' height='600' style='border:none;'></iframe>"))
elif env == "jupyter":
    from IPython.display import HTML, display, clear_output
    clear_output(wait=True)
    print("Open http://127.0.0.1:5000 in your browser.")
    th = threading.Thread(target=app.run, kwargs={"use_reloader": False, "port": 5000, "host": "127.0.0.1"}, daemon=True)
    th.start()
    cache_buster = int(time.time())
    display(HTML(f"<iframe src='http://127.0.0.1:5000/?v={cache_buster}' width='100%' height='600' style='border:none;'></iframe>"))
else:
    print("Starting Flask app on http://127.0.0.1:5000")
    app.run(port=5000, host="127.0.0.1")


* ngrok tunnel: https://lanceted-enoch-judgementally.ngrok-free.dev


/usr/local/lib/python3.12/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
